In [1]:
import requests
from bs4 import BeautifulSoup as BSoup
import pandas as pd

In [2]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine('sqlite:///edgar_idx.db')
conn = engine.connect()
conn.begin()

In [3]:
import sqlite3
conn3 = sqlite3.connect('edgar_idx.db')
cursor=conn3.cursor()

In [4]:
ticker = "NRP"
#cursor.execute('''SELECT * FROM idx WHERE Symbol=?;''', ("ABBV",))
cursor.execute('''SELECT * FROM cik_ticker_name WHERE ticker=?;''',(ticker,))
res=cursor.fetchall()
print(res)
cursor.execute('''SELECT * FROM idx WHERE cik=?;''', (res[0][0],))
recs = cursor.fetchall()
names = list(map(lambda x: x[0], cursor.description))
print(names)
df = pd.DataFrame(data=recs, columns=names)
df['date'] = pd.to_datetime(df['date'])
df.columns
df.size
df.dtypes
#print(type(recs))
print(recs)
#conn3.close()
df[df.type == "10-Q"].to_csv("TestData\\"+ticker.lower()+"_all_10qs.csv", index=None)
df[df.type == "10-K"].to_csv("TestData\\"+ticker.lower()+"_all_10ks.csv", index=None)
#df[df.type == "20-F"].to_csv("TestData\\"+ticker.lower()+"_all_20fs.csv", index=None)

[('1171486', 'NRP', 'Natural Resource Partners LP')]
['cik', 'conm', 'type', 'date', 'path']
[('1171486', 'NATURAL RESOURCE PARTNERS LP', '10-K', '2014-02-28', 'edgar/data/1171486/0001193125-14-077852.txt'), ('1171486', 'NATURAL RESOURCE PARTNERS LP', '4', '2014-02-13', 'edgar/data/1171486/0001140361-14-006818.txt'), ('1171486', 'NATURAL RESOURCE PARTNERS LP', '4', '2014-02-13', 'edgar/data/1171486/0001140361-14-006828.txt'), ('1171486', 'NATURAL RESOURCE PARTNERS LP', '4', '2014-02-13', 'edgar/data/1171486/0001140361-14-006830.txt'), ('1171486', 'NATURAL RESOURCE PARTNERS LP', '4', '2014-02-13', 'edgar/data/1171486/0001140361-14-006831.txt'), ('1171486', 'NATURAL RESOURCE PARTNERS LP', '4', '2014-02-13', 'edgar/data/1171486/0001140361-14-006836.txt'), ('1171486', 'NATURAL RESOURCE PARTNERS LP', '4', '2014-02-13', 'edgar/data/1171486/0001140361-14-006838.txt'), ('1171486', 'NATURAL RESOURCE PARTNERS LP', '4', '2014-02-13', 'edgar/data/1171486/0001140361-14-006841.txt'), ('1171486', 'NA

In [10]:
ticker = "APTV"
req = requests.get("https://www.sec.gov/cgi-bin/browse-edgar?CIK={:s}&owner=exclude&action=getcompany&Find=Search".format(ticker.lower()))
soup = BSoup(req.content, "lxml")
conmTag = soup.find("span", {"class": "companyName"})
if conmTag:
    print(conmTag.text)
atags = soup.findAll("a")
for t in atags:
    if "see all company filings" in t.text:
        print(t.text)

In [5]:
n=-3
df[df.type == "10-Q"].iloc[n]

cik                                         769397
conm                                  AUTODESK INC
type                                          10-Q
date                           2018-06-08 00:00:00
path    edgar/data/769397/0000769397-18-000024.txt
Name: 464, dtype: object

In [6]:
from SecFiling10Q import SecFiling10Q
filing=SecFiling10Q("ADSK")
fname=filing.download(df[df.type == "10-Q"].iloc[n].cik,
                      df[df.type == "10-Q"].iloc[n].conm,
                      df[df.type == "10-Q"].iloc[n].type,
                      df[df.type == "10-Q"].iloc[n].date.strftime("%Y-%m-%f"),
                      df[df.type == "10-Q"].iloc[n].path)
filing.load(fname)
print(filing.getEps())
print(filing.errorLog)

-0.38
['Report date: 2018-04-30 00:00:00, industrial class: SERVICES-PREPACKAGED SOFTWARE [7372]', 'Trying to fall back on extracted instance document.\n', 'Trying to fall back on extracted instance document.\n', 'Trying to fall back on extracted instance document.\n', 'Trying to fall back on extracted instance document.\n', 'Trying to fall back on extracted instance document.\n', 'Trying to fall back on extracted instance document.\n', 'Trying to fall back on extracted instance document.\n', 'Trying to fall back on extracted instance document.\n', 'Trying to fall back on extracted instance document.\n', 'Trying to fall back on extracted instance document.\n', 'Trying to fall back on extracted instance document.\n']


In [12]:
f = open (fname, "r")
## TODO: Do I need to have the whole file in memory?
wholeFile = f.read ()
f.close ()
soup = BSoup (wholeFile, "lxml")

In [14]:
soup.find("document")

<document>
<type>10-Q
<sequence>1
<filename>adsk-4302018x10q.htm
<description>10-Q
<text>
<xbrl>
<?xml version="1.0" encoding="UTF-8"?>
<!--XBRL Document Created with Wdesk from Workiva-->
<!--p:F2C709D8C20655868E620D9A76382ABE,x:21605759409e4478b0d4cc41fba378f4-->
<!-- Document created using Wdesk  -->
<!-- Copyright 2018 Workiva -->
<meta content="text/html" http-equiv="Content-Type"/>
<title>Document</title>
<div style="display:none"><ix:header><ix:hidden><ix:nonfraction contextref="FI2018Q4" decimals="0" id="Fact-4C3A846668575C0EB167E195C6C76100-wk-Fact-4C3A846668575C0EB167E195C6C76100" name="adsk:DebtSecuritiesAvailableforsaleContinuousUnrealizedLossPosition12MonthsorLongerAccumulatedLoss" unitref="usd">0</ix:nonfraction><ix:nonnumeric contextref="FD2019Q1YTD_us-gaap_PlanNameAxis_adsk_EmployeeQualifiedStockPurchasePlan1998EspPlanMember" id="Fact-86BD6957E6F05E2E9EFE873F5A390D1C-wk-Fact-86BD6957E6F05E2E9EFE873F5A390D1C" name="adsk:EmployeeStockPurchasePlanTermofExercisePeriod">P6M<

In [16]:
doc_tag = soup.find_all("document")
xbrlInstance = None
count=0
for tag in doc_tag:
    desc = tag.find("description")
    if desc:
        description = desc.get_text().lower()
        if (description.startswith("idea: xbrl document")):
            if not xbrlInstance:
                print("idea doc found")
                xml_tags = tag.find("xml")
                #self.errorLog.append(xml_tags)
                if xml_tags:
                    xbrlInstance = tag

idea doc found


In [16]:
all_tags = xbrlInstance.findAll()
for tag in all_tags:
    if 'us-gaap:earningspersharebasic' in tag.name:
        print(tag)

<us-gaap:earningspersharebasic contextref="FD2019Q3QTD" decimals="2" id="d3042864e2625-wk-Fact-DAC486E2516F59659312E1E677E66D31" unitref="usdPerShare">-0.11</us-gaap:earningspersharebasic>
<us-gaap:earningspersharebasic contextref="FD2018Q3QTD" decimals="2" id="d3042864e2645-wk-Fact-95FAD29989D15CF39D67B82685601A65" unitref="usdPerShare">-0.55</us-gaap:earningspersharebasic>
<us-gaap:earningspersharebasic contextref="FD2019Q3YTD" decimals="2" id="d3042864e2665-wk-Fact-32314BBBE162585F9DD2615D66AEA3BF" unitref="usdPerShare">-0.67</us-gaap:earningspersharebasic>
<us-gaap:earningspersharebasic contextref="FD2018Q3YTD" decimals="2" id="d3042864e2686-wk-Fact-ED8AA2BB420750BEBA5E2CC8E86B8453" unitref="usdPerShare">-1.79</us-gaap:earningspersharebasic>
<us-gaap:earningspersharebasic contextref="FD2019Q3QTD" decimals="2" id="d3054825e2342-wk-Fact-DAC486E2516F59659312E1E677E66D31" unitref="usdPerShare">-0.11</us-gaap:earningspersharebasic>
<us-gaap:earningspersharebasic contextref="FD2019Q3QTD_

In [13]:
all_10Ks =pd.read_csv("TestData\\nvda_all_10ks.csv", parse_dates=['date'], dtype={'cik':str, 'conm':str, 'type':str,'path':str})
all_10Qs =pd.read_csv("TestData\\nvda_all_10qs.csv", parse_dates=['date'], dtype={'cik':str, 'conm':str, 'type':str,'path':str})


       cik         conm  type       date  \
0  1045810  NVIDIA CORP  10-K 2016-03-17   
1  1045810  NVIDIA CORP  10-K 2017-03-01   
2  1045810  NVIDIA CORP  10-K 2018-02-28   

                                          path  
0  edgar/data/1045810/0001045810-16-000205.txt  
1  edgar/data/1045810/0001045810-17-000027.txt  
2  edgar/data/1045810/0001045810-18-000010.txt   cik             object
conm            object
type            object
date    datetime64[ns]
path            object
dtype: object


In [21]:
import datetime as dt
(dt.datetime(2016,12, 31) - dt.datetime(2018,6, 30)).days

-546

In [32]:
9.35-1.68- 2.11- 3.38

2.1800000000000006

In [13]:
229234 - 45408- 52896- 78351

52579

In [44]:
import pandas as pd
all_10Qs =pd.read_csv("TestData\\nvda_all_10qs.csv", parse_dates=['date'], dtype={'cik':str, 'conm':str, 'type':str,'path':str})


In [50]:
all_10Qs[all_10Qs.date == "2017-05-23"].index[0]

1

In [10]:
from datetime import datetime, timedelta
datetime.now() - timedelta(days=5*365.25)

datetime.datetime(2014, 10, 26, 6, 23, 2, 630704)

In [14]:
currentDate = date(2019,2,1)
oldestDate = datetime(currentDate.year, currentDate.month, currentDate.day) - timedelta(days=5*365.25)
print(oldestDate.isoformat())

2014-01-31T18:00:00


## Candidate keywords  
### Shareholder's equity  
*  FinancialSupportCapitalContributionsMember
*  StatementOfStockholdersEquityAbstract
*  StockholdersEquity
### EPS/Earnings/Profits/Income  
*  EarningsPerShareBasic
*  EarningsPerShareDiluted
*  IncomeLossFromContinuingOperationsPerDilutedShare(?)
*  IncomeLossFromContinuingOperationsPerBasicShare (?)
### Sales/Net sales/net revenue  
*  SalesRevenueNetMember (?)
*  Search for 'Revenue' in income statement (?)
### Others that might be of interest  
*  us-gaap:StockRepurchasedDuringPeriodShares


# Data to extract from filing:
*  Earnings for current Q
*  Earnings for last 12-20 Q
*  Sales
*  Annual EPS for last 4 years
*  ROE (= net income / shareholder's equity)
# Parameters to calculate for each company:
*  Earnings increase: earning for current Q/earnings for same Q prior year
*  Excellency of earnings increase: each of two most recent Q/same Q's prior year + growth rate over last three years
*  Earnings growth acceleration in last 10 Q (calculate 2nd derivative?)
   *  two consecutive Qs of earnings *deceleration* could mean trouble
   *  plot earnings on log-scale for last 12 months to see acceleration
*  Sales increase: 
   *  current Q/prior Q > 25%
   *  *or* Sales growth is accelerating in the last three Q
*  Acceleration of sales growth and earnings growth in last three Q (don't sell if accelerating!)
*  Annual EPS should be increasing in each of the last three years.
*  ROE>17%
*  Stability of Q-EPS over last 3-5 years (plot EPS, fit line through data to determine growth trend, calculate deviation of EPS's from this growth trend (i.e. goodness of fit))

In [2]:
import requests
from bs4 import BeautifulSoup as BSoup
import pandas as pd

In [ ]:
ticker = "NVDA"
## Read in the file
f = open ("1045810_NVIDIA-CORP_10-Q_2017-08-23", "r")
lines = f.readlines ()
f.close ()
ser = pd.Series (lines)
#print (ser)
#this denotes where the actual 10-Q starts. I think the 10-Q part is enclosed by <document> tags
html_idxs = (ser[ser == '<TYPE>10-Q\n'])
#get the line number (== index number):
print ("html_idxs: {}".format(html_idxs.index.values))
#and the corresponding line
print (lines[html_idxs.index.values[0]])
#html_idxs_end = (ser['<\TYPE>' in ser])
#print ("html_idxs_end: {}".format (html_idxs_end.index.values))
#print (lines[html_idxs_end.index.values[0]])

#now I can parse the 10-Q with beautifulsoup:
## Find a way to determine which lines to parse automatically. Really, I only need the html-tag, i.e. lines 55-63 
## (see NVDA_finstat_test.hmtl)
soup = Bsoup ("".join (lines[55:63]), "lxml") #find the end of the section by searching for /Document?
#collect all div-tags for future use
all_divs = soup.find_all ("div")
#find the div-tag that contains 'ITEM 1.&#160;&#160;FINANCIAL STATEMENTS' which indicates the start of the
#income statement's table
#will this work on all filings???

print ("found {} occurrences". format (len(all_divs)))
tables = soup.find_all ("table")
print ("found {} tables".format (len(tables)))
#iterate over the div-tags, not sure if there is a better way. I think everything is is div tags.
flag = False
count = 0
for f in all_divs:
    #find beginning of financial statements
    if (f.get_text().startswith ("ITEM 1.") and "FINANCIAL STATEMENTS" in f.get_text()):
        flag = True
        #print (count)
    if (flag):
        #print (f.get_text())
        if ("three months ended" in f.text.lower()):
            ## At this point, we found the div-tag with the income-statement table
            ## Try to use pandas
            #print((f.find_all('div', text='$')))
            for ff in f.find_all('div',text='$'):
                ff.decompose()
            ## Almost! Just need to get rid of the '$', which are offsetting the affected rows by several columns
            ## This removes the '$' (use re.compile for other currency symbols?), now what about the paren's denoting negative numbers?
            tableDf=pd.read_html(str(f))
            print(tableDf)
            ## Manual method
            trs = f.find_all("tr")
            #print (len(trs))
            #print(trs[2].get_text())
            for t in trs:
                tlist = [item.get_text() for item in t.find_all("td")]
                print(" || ".join(tlist))
            #print (trs)
            break
        #print ("\n")
    count += 1



### See these urls for downloading the appropriate files, and parsing XBRL with Python/BeautifulSoup
*  https://www.codeproject.com/Articles/1227765/Parsing-XBRL-with-Python
*  https://www.slideshare.net/afalk42/xbrl-us-altova-webinar
### Taxonomy:
*  http://www.xbrlsite.com/LinkedData/BrowseObjectsByType_HTML.aspx?Type=%5BConcept%5D&Submit=Submit
*  ~~I think I want the instance xml file for each filing,~~ though that \*.txt file I've been downloading so far contains *everything* (I think). Could use that as well, the benefits are that the name of this file is standardized and I already have the machinery to download it in place, but the trade-off is that it's the biggest file for each filing. 
   *  The instance document can be found in the \*.txt file; it has the `<description>` XBRL INSTANCE DOCUMENT. Inside the description-tag is the `<XBRL>`, which contains the XBRL-data for the instance document. Note that there are several XBRL- tags containing the various XBRL documents (schema, extension, linkbases, ...).  
*  Search for the tags 'us-gaap:\*', they contain the items for the financial statements.
*  Also, Wikipedia has a good reference on the components of XBRL: 
   *  https://en.wikipedia.org/wiki/XBRL
### To do:
-  Figure out which taxonomy items I want/need, and all their permutations
-  Determine the date on each item
   -  Each us-gaap:\* - tag has a context-ref (e.g. "FI2017Q4"). Find the corresponding context (in the instance document, below the us-gaap - tags):
      ```
      <xbrli:context id="FI2017Q4">
		<xbrli:entity>
			<xbrli:identifier scheme="http://www.sec.gov/CIK">0001045810</xbrli:identifier>
		</xbrli:entity>
		<xbrli:period>
			<xbrli:instant>2017-01-29</xbrli:instant>
		</xbrli:period>
	</xbrli:context>
    ```  
### Interesting classifiers:
-  unitref=


In [6]:
g=open("SECDATA\\AUTODESK INC\\769397_AUTODESK INC_10-Q_2018-08-000000", "r")
wholeFile = g.read ()
g.close ()
soup = BSoup (wholeFile, "lxml")
#print(soup)

In [7]:
sec_header = soup.find("sec-header")
print(sec_header)
doc_tag = soup.find_all("document")
#print(doc_tag)

None


In [32]:
import os
path=os.path.join(os.getcwd(), os.path.join("SECDATA", "AUTODESK INC"))
print(path)
print(os.listdir(path))

C:\Users\Carola\jupyternbs\SECDATA\AUTODESK INC
['769397_AUTODESK INC_10-K_2013-03-000000', '769397_AUTODESK INC_10-K_2014-03-000000', '769397_AUTODESK INC_10-K_2015-03-000000', '769397_AUTODESK INC_10-K_2016-03-000000', '769397_AUTODESK INC_10-K_2017-03-000000', '769397_AUTODESK INC_10-K_2018-03-000000', '769397_AUTODESK INC_10-Q_2013-06-000000', '769397_AUTODESK INC_10-Q_2013-09-000000', '769397_AUTODESK INC_10-Q_2013-12-000000', '769397_AUTODESK INC_10-Q_2014-06-000000', '769397_AUTODESK INC_10-Q_2014-09-000000', '769397_AUTODESK INC_10-Q_2014-12-000000', '769397_AUTODESK INC_10-Q_2015-05-000000', '769397_AUTODESK INC_10-Q_2015-08-000000', '769397_AUTODESK INC_10-Q_2015-12-000000', '769397_AUTODESK INC_10-Q_2016-06-000000', '769397_AUTODESK INC_10-Q_2016-08-000000', '769397_AUTODESK INC_10-Q_2016-12-000000', '769397_AUTODESK INC_10-Q_2017-05-000000', '769397_AUTODESK INC_10-Q_2017-08-000000', '769397_AUTODESK INC_10-Q_2017-12-000000', '769397_AUTODESK INC_10-Q_2018-06-000000', '7693